<a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"><img src = "https://cdn.torontolife.com/wp-content/uploads/2016/10/toronto-skyline-803x603-1476458932.jpg" width = 140, align = "center"></a>

<h1 align=center><font size = 5>Clustering Neighborhoods in Toronto</font></h1>

### Autor: Patrick Franco Alves

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Install and load the required packages</a>

2. <a href="#item2">Latitude and longitude of each neighborhood in Toronto</a>

3. <a href="#item3">Clustering neighborhoods in Toronto</a>

4. <a href="#item4">Filtering the boroughs with word Toronto</a>

5. <a href="#item5">Define Foursquare credentials</a>  
    
6. <a href="#item6">Creating dataframe with Toronto foursquare venues</a>  
    
7. <a href="#item7">One hot enconding</a>  
    
8. <a href="#item8">Only top 20 venues by neighborhood</a>  
    
9. <a href="#item9">Visualizing the resulting, cluster by cluster</a>  
</font>
</div>


## 1. Install and load the required packages

In [36]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup

In [37]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [38]:
table = soup.find('table', class_='wikitable')
codes = {'PostalCode':[], 'Borough':[], 'Neighborhood':[]}
if table:
    rows = table.findAll('tr')
    for tr in rows[1:]:
        tds = tr.findAll('td')
        code = tds[0].text.strip()
        borough = tds[1].text.strip()
        nhood = tds[2].text.strip()
        if borough != 'Not assigned':
            codes['PostalCode'].append(code)
            codes['Borough'].append(borough)
            if nhood == 'Not assigned': nhood = borough
            codes['Neighborhood'].append(nhood)

# make a datafram from our dictionary
codes_df = pd.DataFrame(codes)

# combine rows with the same postal code into one row with the neighborhoods separated with a comma 
codes_df =  codes_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()

In [39]:
print(codes_df.shape)
codes_df.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 2. Latitude and longitude of each neighborhood in Toronto

In [40]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data

print('Data downloaded!')

Data downloaded!


In [41]:
geocoordes = pd.read_csv('https://cocl.us/Geospatial_data')
geocoordes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
# join two dataframes into one
to_codes = codes_df.join(geocoordes.set_index('Postal Code'), on='PostalCode')

to_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3. Clustering neighborhoods in Toronto

#### Using folium

In [43]:
import folium
# let's use the latitude and longitude of Central Toronto to create map
center_lat = to_codes[toronto_codes['Borough'] == 'Central Toronto'].iloc[0].Latitude
center_lng = to_codes[toronto_codes['Borough'] == 'Central Toronto'].iloc[0].Longitude

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[center_lat, center_lng], zoom_start=11)

# add markers to map
for lat, lng, borough, code in zip(to_codes['Latitude'], to_codes['Longitude'], to_codes['Borough'], to_codes['PostalCode']):
    label = '{}, {}'.format(code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# 4. Filtering the boroughs with word 'Toronto'

In [44]:
toronto_data = toronto_codes[to_codes['Borough'].str.find('Toronto') != -1].reset_index(drop=True)
print('Number of boroughs is {}'.format(len(toronto_data['Borough'].unique())))
print('Number of postcode areas is {}'.format(toronto_data.shape[0]))
toronto_data.head()

Number of boroughs is 4
Number of postcode areas is 38


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Let's visualize postcode areas of that boroughs with word Toronto.

In [45]:
# let's use the latitude and longitude of Downtown Toronto to create map
center_lat = to_codes[to_codes['Borough'] == 'Downtown Toronto'].iloc[0].Latitude
center_lng = to_codes[to_codes['Borough'] == 'Downtown Toronto'].iloc[0].Longitude

# create map 
map_toronto2 = folium.Map(location=[center_lat, center_lng], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

# 5. Define Foursquare Credentials

In [47]:
CLIENT_ID = 'USC23TI2Y3K0WLFYXPJUXXEJVZ4NUEMT3CR5T3ZCNTJ3IB55' # your Foursquare ID
CLIENT_SECRET = 'U2QVNANSIRP4OBT5MWECVSUP0VR1PVEQLDKY5S4HY4CNCP0S' # your Foursquare Secret
VERSION = '20191013'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: USC23TI2Y3K0WLFYXPJUXXEJVZ4NUEMT3CR5T3ZCNTJ3IB55
CLIENT_SECRET:U2QVNANSIRP4OBT5MWECVSUP0VR1PVEQLDKY5S4HY4CNCP0S


#### Explore neighborhoods in Toronto (that in our case are postcode areas with one or several neighborhoods)

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'PostalCode', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

# 6. Creating dataframe with Toronto foursquare venues

In [49]:
venues = getNearbyVenues(
    names=toronto_data['PostalCode'],
    latitudes=toronto_data['Latitude'],
    longitudes=toronto_data['Longitude'] )

In [50]:
print(venues.shape)
venues.head()

(1173, 7)


,PostalCode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Display the number of venues were returned for each neighborhood.

In [51]:
toronto_venues.groupby('PostalCode').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M4E,4,4,4,4,4,4
M4K,42,42,42,42,42,42
M4L,22,22,22,22,22,22
M4M,41,41,41,41,41,41
M4N,3,3,3,3,3,3
M4P,8,8,8,8,8,8
M4R,22,22,22,22,22,22
M4S,35,35,35,35,35,35
M4T,3,3,3,3,3,3


In [52]:
# unique categories from all returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


# 7. One hot enconding 

In [53]:
# one hot encoding
venues_hot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_hot['PostalCode'] = venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [venues_hot.columns[-1]] + list(venues_hot.columns[:-1])
venues_hot = venues_hot[fixed_columns]

print(venues_hot.shape)
venues_hot.head()

(1173, 214)


,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [54]:
to_group = venues_hot.groupby('PostalCode').mean().reset_index()

print(to_group.shape)
to_group.head()

(38, 214)


,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.00000,0.0,0.25000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
1,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.02381,0.0,0.0,...,0.00000,0.0,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,0.02381
2,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
3,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.04878,0.0,0.0,...,0.02439,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.02439
4,M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


# 8. Only the 20 venues by neighborhood

Define a function to sort the venues in descending order.

In [55]:
def get_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe with the top 10 venues.

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues = pd.DataFrame(columns=columns)
neighborhoods_venues['PostalCode'] = to_group['PostalCode']

for ind in np.arange(to_group.shape[0]):
    neighborhoods_venues.iloc[ind, 1:] = get_venues(to_group.iloc[ind, :], num_top_venues)

neighborhoods_venues.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Trail,Pub,Neighborhood,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,M4L,Park,Sandwich Place,Brewery,Intersection,Italian Restaurant,Steakhouse,Fish & Chips Shop,Fast Food Restaurant,Liquor Store,Burger Joint
3,M4M,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Convenience Store,Brewery,Sandwich Place,Cheese Shop
4,M4N,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [57]:
from sklearn.cluster import KMeans 

# set number of clusters
kclusters = 5

to_group_cluster = to_group.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_group_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 0, 1, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [58]:
# add clustering labels
neighborhoods_venues.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues.set_index('PostalCode'), on='PostalCode')

print(toronto_merged.shape)
toronto_merged.head()

(38, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Neighborhood,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Brewery,Intersection,Italian Restaurant,Steakhouse,Fish & Chips Shop,Fast Food Restaurant,Liquor Store,Burger Joint
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Convenience Store,Brewery,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


## 9. Visualizing the resulting, cluster by cluster

In [59]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[center_lat, center_lng], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Health Food Store,Trail,Pub,Neighborhood,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,"The Beaches West, India Bazaar",Park,Sandwich Place,Brewery,Intersection,Italian Restaurant,Steakhouse,Fish & Chips Shop,Fast Food Restaurant,Liquor Store,Burger Joint
3,Studio District,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Convenience Store,Brewery,Sandwich Place,Cheese Shop
5,Davisville North,Gym,Breakfast Spot,Park,Clothing Store,Food & Drink Shop,Hotel,Dog Run,Sandwich Place,Discount Store,Event Space
6,North Toronto West,Sporting Goods Shop,Clothing Store,Coffee Shop,Gym / Fitness Center,Gift Shop,Mexican Restaurant,Diner,Dessert Shop,Cosmetics Shop,Park
7,Davisville,Dessert Shop,Sandwich Place,Coffee Shop,Pizza Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Thai Restaurant,Pharmacy
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,Health & Beauty Service,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Bagel Shop,Liquor Store,Fried Chicken Joint
11,"Cabbagetown, St. James Town",Coffee Shop,Park,Restaurant,Café,Bakery,Grocery Store,Pub,Pizza Place,Italian Restaurant,Indian Restaurant
12,Church and Wellesley,Coffee Shop,Gym,Burger Joint,Restaurant,Men's Store,Gay Bar,Gastropub,Ethiopian Restaurant,Bookstore,Indian Restaurant


#### Cluster 2

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park, Summerhill East",Gym,Summer Camp,Playground,Comic Shop,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 3

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 4

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,Health & Beauty Service,Garden,Home Service,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 5

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,Park,Playground,Trail,Building,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
23,"Forest Hill North, Forest Hill West",Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
